In [1]:
#import libraries
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
#from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

#import requests # library to handle requests
#from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
#import matplotlib.cm as cm
#import matplotlib.colors as colors

# import k-means from clustering stage
#from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
#import folium # map rendering library

#!conda install -c conda-forge beautifulsoup4

print('Libraries imported.')

Libraries imported.


In [2]:
#download data
codes_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
#website_url = requests.get(codes_url).text
#!wget -q -O 'postal_codes.json' https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
#print('Data downloaded!')

### Load html into dataframe

In [3]:
#with open('postal_codes.json') as json_data:
#    html = json.load(json_data)
import requests 
from bs4 import BeautifulSoup
import lxml
from io import StringIO 
import lxml.html
from urllib.request import urlopen


data = urlopen(codes_url).read();

doc = lxml.html.document_fromstring(str(data))
el = doc.xpath("//table[@class='wikitable sortable']//text()")
#el
ret = '\\n'
nodes = []
for t in el:
    if t =='\\n' :
        continue
    elif t.endswith(ret):
        nodes.append(t.replace(ret,''))
    else: 
        nodes.append(t)
nodes
i=0
postcodes = []
boroughs = []
neighborhoods = []
for t in nodes:
    if i % 3 == 0:
        postcodes.append(t)
    elif i % 3 == 1:
        boroughs.append (t)
    else:
        neighborhoods.append(t)
    i+=1

column_names = ['Postcodes', 'Borough', 'Neighborhood'] 

# instantiate the dataframe
codes = pd.DataFrame()
codes['PostalCode']=postcodes
codes['Borough']=boroughs
codes['Neighborhood']=neighborhoods
#codes


### Filter out 'Not assigned' Borough and first line headers

In [4]:
df = codes[codes.Borough != 'Not assigned']
df = df[df.Borough != 'Borough']
print(df.shape)


(212, 3)


### Reassign Neighborhood to Borough if Neighborhood is  'Not assigned'

In [5]:
df['Neighborhood'][df.Neighborhood == 'Not assigned']=df.Borough[df.Neighborhood == 'Not assigned']
#print(df.shape)
#df






###  Create  Merged Neighborhood for repeating PostalCode  and remove duplicate rows; print shape and final dataframe df1

In [6]:


df['Neighborhood'] = df[['PostalCode','Borough','Neighborhood']].groupby(['PostalCode','Borough'])['Neighborhood'].transform(lambda x: ','.join(x))
df1 = df[['PostalCode','Borough','Neighborhood']].drop_duplicates()

print('\n DataFrame Shape:', df1.shape, '\n\n')

#df1



 DataFrame Shape: (103, 3) 




### Load location data from file

In [7]:
!wget -q -O 'postal_codes.csv' https://cocl.us/Geospatial_data

In [15]:
df_f=pd.read_csv('postal_codes.csv')
df_f=df_f.rename(columns={'Postal Code': 'PostalCode'})



### merge two dataframe by making index from the column of join

In [16]:
#print(df_f)
df2=df1.join(df_f.set_index('PostalCode'), on='PostalCode')
df2

,PostalCode,Borough,Neighborhood,Latitude,Longitude
3,M3A,North York,Parkwoods,43.753259,-79.329656
4,M4A,North York,Victoria Village,43.725882,-79.315572
5,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
7,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
9,M7A,Queen\'s Park,Queen\'s Park,43.662301,-79.389494
11,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
12,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
15,M3B,North York,Don Mills North,43.745906,-79.352188
16,M4B,East York,"Woodbine Gardens,Parkview Hill",43.706397,-79.309937
18,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937
